In [8]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


data_dir = '../data/_datasets/'
data_file = 'seq_dg_cluster_final_230607_v2.csv'
data_file = os.path.join(data_dir, data_file)
df = pd.read_csv(data_file, sep='\t', low_memory=False)
df.head(3)

,complex_id,dataset_id,UniProt,protein_sequence,nucleotide_sequence,protein_mutation,nucleic_acid_type,nucleic_acid_type_new,Kd,dG,...,cluster_na_50_index,cluster_na_50_number,cluster_na_50_member,protein_na_pair_index,protein_50_na_50,protein_50_na_50_number,cluster_protein_na_pair_index,dataset_id_pdb_id_protein_index,protein_species,na_jobid
0,0,pnat_143,P04150,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,CGCCTCCGGGAGAGCT,wild,ssDNA,ssDNA,3.6e-07,-8.79,...,Cluster_1008,1,ssDNA0,protein_6_ssDNA0,protein_6_ssDNA0,1,cluster_pair_0,pnat_143_3fyl_protein_6,Rattus norvegicus,DNA_0022
1,1,pnat_13395,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCTTCCCT,wild,DNA,ssDNA,1.83e-06,-7.83,...,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA1,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13395_1jmc_protein_12,Homo sapiens,DNA_0225
2,2,pnat_13396,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CCCTTCTTTCTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCT...,wild,DNA,ssDNA,1.39e-06,-7.99,...,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA2,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13396_1jmc_protein_12,Homo sapiens,DNA_0226


In [4]:
df.keys()

Index(['Name', 'protein_seq', 'Steric_parameter', 'polarizability', 'volume',
       'hydrophobicity', 'isoelectric point', 'Helix probability',
       'Sheet probability', 'CHAM820101', 'KLEP840101', 'RADA880108',
       'b2btool_backbone', 'b2btool_sidechain', 'provean',
       'fIDPnn_disorder_binary', 'fIDPnn_disorder_propensity',
       'fIDPnn_protein_binding_propensity', 'fIDPnn_DNA_binding_propensity',
       'fIDPnn_RNA_binding_propensity', 'fIDPnn_linker_propensity',
       'fIDPnn_disordered_bind_protein', 'fIDPnn_disordered_not_bind_protein',
       'fIDPnn_disordered_bind_DNA', 'fIDPnn_disordered_not_bind_DNA',
       'fIDPnn_disordered_bind_RNA', 'fIDPnn_disordered_not_bind_RNA',
       'fIDPnn_disordered_linker_protein',
       'fIDPnn_not_disordered_linker_protein', 'spot_1d_ASA', 'spot_1d_HseU',
       'spot_1d_HseD', 'spot_1d_CN', 'spot_1d_Psi', 'spot_1d_Phi',
       'spot_1d_Theta', 'spot_1d_Tau', 'spot_1d_P3C', 'spot_1d_P3E',
       'spot_1d_P3H', 'spot_1d_P8C', 'sp

In [4]:
### other features of NC

#9种物理化学性质:Steric_parameter,polarizability,volume,hydrophobicity,isoelectric point	Helix probability,Sheet probability,CHAM820101,KLEP840101,RADA880108;
#残基的主链或者侧链灵活性:b2btool_backbone,b2btool_sidechain;
#残基保守性:provean;
#残基是否 disorder，以及是否和蛋白,核酸结合:fIDPnn为前缀的列;
#残基的溶表、二级结构和扭转角:spot_1d为前缀的列;

# for i in range(len(df)):
#     df.loc[i]['nucleic_acid_type_new']
# df.loc[12]['nucleic_acid_type_new']

na_jobid = df.loc[0]['na_jobid']
print(na_jobid)

nucleotide_sequence = df.loc[0]['nucleotide_sequence']
print(len(nucleotide_sequence))

data_dir = '../data/nc_data'
dna_feat_file = os.path.join(data_dir, na_jobid, '{}_EIIP.txt'.format(na_jobid))
dna_chemi_df = pd.read_csv(dna_feat_file, sep='\t')
print(len(dna_chemi_df))
dna_chemi_df.head()

DNA_0022
16
16


,seq,EIIP
0,C,0.1340
1,G,0.0806
2,C,0.1340
3,C,0.1340
4,T,0.1335


In [5]:
not_equal_list = []
for i in range(len(df)):
    if 'DNA' in df.loc[i]['nucleic_acid_type_new']:
        na_jobid = df.loc[i]['na_jobid']
        nucleotide_sequence = df.loc[i]['nucleotide_sequence'].replace('|', '')


        data_dir = '../data/nc_data'
        dna_feat_file = os.path.join(data_dir, na_jobid, '{}_EIIP.txt'.format(na_jobid))
        if os.path.exists(dna_feat_file):
            
            dna_chemi_df = pd.read_csv(dna_feat_file, sep='\t')

            if (len(nucleotide_sequence) != len(dna_chemi_df)):
                not_equal_list.append(i)
print(not_equal_list)

[]


In [9]:
### chemistry features of protein

#9种物理化学性质:
#   Steric_parameter,polarizability,volume,hydrophobicity,isoelectric point,
#   elix probability,Sheet probability,CHAM820101,KLEP840101,RADA880108;
#残基的主链或者侧链灵活性:b2btool_backbone,b2btool_sidechain;
#残基保守性:provean;
#残基是否 disorder，以及是否和蛋白,核酸结合:fIDPnn为前缀的列;
#残基的溶表、二级结构和扭转角:spot_1d为前缀的列;

data_dir = '../data/protein_all_feas/'
data_file = 'protein_607.txt'
data_file = os.path.join(data_dir, data_file)
df = pd.read_csv(data_file, sep='\t')
pd.set_option('display.max_columns', len(df.columns))
df.head(3)

,Name,protein_seq,Steric_parameter,polarizability,volume,hydrophobicity,isoelectric point,Helix probability,Sheet probability,CHAM820101,KLEP840101,RADA880108,b2btool_backbone,b2btool_sidechain,provean,fIDPnn_disorder_binary,fIDPnn_disorder_propensity,fIDPnn_protein_binding_propensity,fIDPnn_DNA_binding_propensity,fIDPnn_RNA_binding_propensity,fIDPnn_linker_propensity,fIDPnn_disordered_bind_protein,fIDPnn_disordered_not_bind_protein,fIDPnn_disordered_bind_DNA,fIDPnn_disordered_not_bind_DNA,fIDPnn_disordered_bind_RNA,fIDPnn_disordered_not_bind_RNA,fIDPnn_disordered_linker_protein,fIDPnn_not_disordered_linker_protein,spot_1d_ASA,spot_1d_HseU,spot_1d_HseD,spot_1d_CN,spot_1d_Psi,spot_1d_Phi,spot_1d_Theta,spot_1d_Tau,spot_1d_P3C,spot_1d_P3E,spot_1d_P3H,spot_1d_P8C,spot_1d_P8S,spot_1d_P8T,spot_1d_P8H,spot_1d_P8G,spot_1d_P8I,spot_1d_P8E,spot_1d_P8B,spot_1d_SS3_C_onehot,spot_1d_SS3_E_onehot,spot_1d_SS3_H_onehot,spot_1d_SS8_C_onehot,spot_1d_SS8_S_onehot,spot_1d_SS8_T_onehot,spot_1d_SS8_H_onehot,spot_1d_SS8_G_onehot,spot_1d_SS8_I_onehot,spot_1d_SS8_E_onehot,spot_1d_SS8_B_onehot
0,protein_607,M,2.35,0.22,4.43,1.23,5.71,0.38,0.32,0.221,0.0,1.27,0.6943,0.48790,-0.137,0,0.271,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,179.922790,2.378077,8.052862,10.982017,136.46457,-85.868065,111.14806,-156.65440,0.999992,0.000005,0.000003,0.999995,2.018500e-09,0.000001,0.000001,7.637209e-08,1.542764e-19,0.000002,2.051850e-07,1,0,0,1,0,0,0,0,0,0,0
1,protein_607,V,3.67,0.14,3.00,1.22,6.02,0.27,0.49,0.140,0.0,1.09,0.6896,0.52700,0.095,1,0.418,0.898,0.488,0.437,0.460,1,0,0,1,0,1,0,1,121.383600,4.397138,7.323887,13.057035,129.32872,-90.190895,110.71167,-159.64256,0.924757,0.051125,0.024118,0.905302,2.640741e-04,0.026059,0.012514,4.904244e-03,3.232941e-07,0.029696,2.126109e-02,1,0,0,1,0,0,0,0,0,0,0
2,protein_607,G,0.00,0.00,0.00,0.00,6.07,0.13,0.15,0.000,0.0,-0.41,0.6838,0.45935,-0.964,1,0.458,0.814,0.488,0.418,0.431,1,0,0,1,0,1,0,1,56.278915,2.961766,9.015953,12.897865,179.27463,-80.004330,117.83238,98.48316,0.947472,0.022580,0.029948,0.731048,1.633566e-01,0.055038,0.016526,1.158190e-02,6.073952e-06,0.018748,3.695121e-03,1,0,0,1,0,0,0,0,0,0,0


In [12]:
[x for x in df.keys() if 'spot_1d' in x and 'onehot' not in x]

['spot_1d_ASA',
 'spot_1d_HseU',
 'spot_1d_HseD',
 'spot_1d_CN',
 'spot_1d_Psi',
 'spot_1d_Phi',
 'spot_1d_Theta',
 'spot_1d_Tau',
 'spot_1d_P3C',
 'spot_1d_P3E',
 'spot_1d_P3H',
 'spot_1d_P8C',
 'spot_1d_P8S',
 'spot_1d_P8T',
 'spot_1d_P8H',
 'spot_1d_P8G',
 'spot_1d_P8I',
 'spot_1d_P8E',
 'spot_1d_P8B']

In [9]:
diso_keys = [it for it in df.keys() if 'fIDPnn' in it]
print(len(diso_keys))
df[diso_keys]

14


,fIDPnn_disorder_binary,fIDPnn_disorder_propensity,fIDPnn_protein_binding_propensity,fIDPnn_DNA_binding_propensity,fIDPnn_RNA_binding_propensity,fIDPnn_linker_propensity,fIDPnn_disordered_bind_protein,fIDPnn_disordered_not_bind_protein,fIDPnn_disordered_bind_DNA,fIDPnn_disordered_not_bind_DNA,fIDPnn_disordered_bind_RNA,fIDPnn_disordered_not_bind_RNA,fIDPnn_disordered_linker_protein,fIDPnn_not_disordered_linker_protein
0,0,0.271,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0
1,1,0.418,0.898,0.488,0.437,0.460,1,0,0,1,0,1,0,1
2,1,0.458,0.814,0.488,0.418,0.431,1,0,0,1,0,1,0,1
3,1,0.477,0.862,0.488,0.409,0.368,1,0,0,1,0,1,0,1
4,1,0.517,0.862,0.416,0.333,0.384,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,1,0.327,0.972,0.488,0.425,0.800,1,0,0,1,0,1,1,0
337,1,0.333,1.000,0.488,0.413,0.644,1,0,0,1,0,1,1,0
338,1,0.409,0.943,0.416,0.334,0.555,1,0,0,1,0,1,1,0
339,1,0.421,0.928,0.416,0.330,0.593,1,0,0,1,0,1,1,0
